In [1]:
!python -m pip install python-dotenv --quiet
!python -m pip install awscli --quiet
!python -m pip install gym stable-baselines3[extra]  sb3-contrib boto3 --quiet


In [2]:
%load_ext dotenv
%dotenv env

In [3]:
!aws s3 sync experiments s3://dissertation-data-dmiller/experiments

upload: experiments/q2/dqn_pred_cnn/2023-07-20_09-18-23/checkpoints/rl_model_1800000_steps.zip to s3://dissertation-data-dmiller/experiments/q2/dqn_pred_cnn/2023-07-20_09-18-23/checkpoints/rl_model_1800000_steps.zip
upload: experiments/q2/dqn_pred_cnn/2023-07-20_09-18-23/checkpoints/rl_model_2400000_steps.zip to s3://dissertation-data-dmiller/experiments/q2/dqn_pred_cnn/2023-07-20_09-18-23/checkpoints/rl_model_2400000_steps.zip
upload: experiments/q2/dqn_pred_cnn/2023-07-20_09-18-23/checkpoints/rl_model_4800000_steps.zip to s3://dissertation-data-dmiller/experiments/q2/dqn_pred_cnn/2023-07-20_09-18-23/checkpoints/rl_model_4800000_steps.zip
upload: experiments/q2/dqn_pred_cnn/2023-07-20_09-18-23/checkpoints/rl_model_6000000_steps.zip to s3://dissertation-data-dmiller/experiments/q2/dqn_pred_cnn/2023-07-20_09-18-23/checkpoints/rl_model_6000000_steps.zip
upload: experiments/q2/dqn_pred_cnn/2023-07-20_09-18-23/checkpoints/rl_model_5400000_steps.zip to s3://dissertation-data-dmiller/experim

In [4]:
!aws s3 sync rl_evaluation s3://dissertation-data-dmiller/rl_evaluation

In [5]:
# %load rl_constant.py
FEATURE_COLUMNS = [
    
    "user_count",
    "project_count", 
    "country_count", 
    "date_hour_sin", 
    "date_hour_cos",
    "date_minute_sin",
    "date_minute_cos",
    
    "session_30_count",
    "session_5_count",
    "cum_session_event",
    "cum_session_time",
    "expanding_click_average",
   
    "cum_platform_time",
    "cum_platform_event",
    "cum_projects",
    "average_event_time",
    "delta_last_event",
    
    "rolling_session_time",
    "rolling_session_events",
    "rolling_session_gap",
    "previous_session_time",
    "previous_session_events",
]



METADATA = [
    "user_id",
    "session_30_count_raw",
    "cum_platform_event_raw",
    "cum_platform_time_raw",
    "cum_session_time_raw",
    "cum_session_event_raw",
    "date_time"
]

RL_STAT_COLS = [
    'session_size',
    'session_minutes',
    'size_cutoff',
    'time_cutoff',
    'reward'
]

PREDICTION_COLS = [
    "label",
    "pred"
]

LOAD_COLS = list(set(FEATURE_COLUMNS + METADATA + RL_STAT_COLS + PREDICTION_COLS))

In [6]:
# %load policy_list.py
POLICY_LIST = [
    {
        "algo": "dqn_pred_cnn",
        "run_date": "2023-06-13_16-11-42"
    }
]

In [7]:
# %load policies/cnn_policy
from typing import Dict, List, Type, Union

import gym
import torch
from gym import spaces
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.dqn.policies import DQNPolicy
import torch.nn.functional as F
from torch import nn
import logging
global logger
logger = logging.getLogger(__name__)

class CustomConv1dFeatures(BaseFeaturesExtractor):
    
    @classmethod
    def setup_sequences_features(cls, n_sequences, n_features):
        cls.n_sequences = n_sequences
        cls.n_features = n_features
        
    
    def __init__(self, observation_space: spaces.Box, features_dim=24):
        super().__init__(observation_space, features_dim)
        
        
        self.cnn_1 = nn.Sequential(
            nn.Conv1d(self.n_features, self.n_features*2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*2),
            nn.ELU(),
            
            nn.Conv1d(self.n_features*2, self.n_features*2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*2),
            nn.ELU(),
            
            nn.Conv1d(self.n_features*2, self.n_features*2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*2),
            nn.ELU()
            
        )
        
        self.conv_1_reshape = nn.Conv1d(
            self.n_features,
            self.n_features*2,
            kernel_size=1,
            padding=0
        
        )
        
        self.a_pool_1 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.cnn_bottleneck_wide = nn.Sequential(
            nn.Conv1d(self.n_features*2, self.n_features*4, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*4),
            nn.ELU(),
            
            nn.Conv1d(self.n_features*4, self.n_features*4, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*4),
            nn.ELU(),
            
            nn.Conv1d(self.n_features*4, self.n_features*4, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*4),
            nn.ELU()   
        )
        
        self.conv_2_reshape = nn.Conv1d(
            self.n_features*2,
            self.n_features*4,
            kernel_size=1,
            padding=0
        )
        
        
        self.cnn_bottleneck_narrow = nn.Sequential(
            nn.Conv1d(self.n_features*4, self.n_features*2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*2),
            nn.ELU(),
            
            nn.Conv1d(self.n_features*2, self.n_features*2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*2),
            nn.ELU(),
            
            nn.Conv1d(self.n_features*2, self.n_features*2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*2),
            nn.ELU()
        )
        
        self.conv_3_reshape = nn.Conv1d(
            self.n_features*4,
            self.n_features*2,
            kernel_size=1,
            padding=0
        )
        
        self.downsample = nn.Sequential(
            nn.Conv1d(self.n_features*2, self.n_features, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features),
            nn.ELU(),
            
            nn.Conv1d(self.n_features, self.n_features, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features),
            nn.ELU(),
            
            nn.Conv1d(self.n_features, self.n_features, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features),
            nn.ELU()
        )
        
        self.conv_4_reshape = nn.Conv1d(
            self.n_features*2,
            self.n_features,
            kernel_size=1,
            padding=0
        )
                
        self.down_max = nn.Sequential(
            nn.Conv1d(self.n_features, self.n_features // 2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features // 2),
            nn.ELU(),
            
            nn.Conv1d(self.n_features // 2, self.n_features // 2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features // 2),
            nn.ELU(),
            
            nn.Conv1d(self.n_features // 2, self.n_features // 2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features // 2),
            nn.ELU(),
        )
        
        
        self.mpool_flat = nn.Sequential(
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Flatten()
        )
        self.down_max_reshape = nn.Conv1d(
            self.n_features,
            self.n_features // 2,
            kernel_size=1,
            padding=0
        )
        
        with torch.no_grad():
            sample_tensor = torch.zeros((1, self.n_features, self.n_sequences))
            sample_tensor = self.cnn_1(sample_tensor) + self.conv_1_reshape(sample_tensor)
            sample_tensor = self.a_pool_1(sample_tensor)
            sample_tensor = self.cnn_bottleneck_wide(sample_tensor) + self.conv_2_reshape(sample_tensor)
            sample_tensor = self.cnn_bottleneck_narrow(sample_tensor) + self.conv_3_reshape(sample_tensor)
            sample_tensor = self.downsample(sample_tensor) + self.conv_4_reshape(sample_tensor)
            sample_tensor = self.down_max(sample_tensor) + self.down_max_reshape(sample_tensor)
            mpool_flat_out = self.mpool_flat(sample_tensor)
            linear_in = mpool_flat_out.shape[1]
            self.final_out_linear = nn.Sequential(

                nn.Linear(linear_in, features_dim),
                nn.ELU()
            )

    def forward(self, obs):
        obs_cnn_1 = self.cnn_1(obs) + self.conv_1_reshape(obs)
        obs_cnn_1 = self.a_pool_1(obs_cnn_1)
        
        obs_cnn_2 = self.cnn_bottleneck_wide(obs_cnn_1) + self.conv_2_reshape(obs_cnn_1) 
        obs_cnn_3 = self.cnn_bottleneck_narrow(obs_cnn_2) + self.conv_3_reshape(obs_cnn_2)
        obs_cnn_4 = self.downsample(obs_cnn_3) + self.conv_4_reshape(obs_cnn_3)
        obs_cnn_5 = self.down_max(obs_cnn_4) + self.down_max_reshape(obs_cnn_4)
        
        mpool_flat_out = self.mpool_flat(obs_cnn_5)
        
        return self.final_out_linear(mpool_flat_out)
        

In [8]:
# %load environment_q2
import gym
import numpy as np
from scipy.stats import norm

MAX_EVAL_SIZE = 75
from bisect import bisect_left

class CitizenScienceEnvQ2(gym.Env):
    
    metadata = {'render.modes': ['human']}
    
    def __init__(self, dataset, out_features, n_sequences, params=None):
        """
        trajectories: dictionary of user_id to their respective trajectories.
        n_sequences: number of sequences used for preprocessing.
        n_features: number of features used for preprocessing.
        """
        super(CitizenScienceEnvQ2, self).__init__()
        self.dataset = dataset
        self.unique_sessions = self.dataset[['user_id', 'session_30_count_raw']].drop_duplicates()
        self.n_sequences = n_sequences
        self.current_session = None
        self.current_session_index = 0
        self.reward = 0
        self.n_sequences = n_sequences
        self.out_features = out_features
        
        max_session_size = self.dataset['session_size'].max()
        
        self.action_space = gym.spaces.Discrete(4)
        self.observation_space = gym.spaces.Box(low=-1, high=91, shape=(len(out_features) + 3, n_sequences + 1), dtype=np.float32)
        self.episode_bins = []
        self.exp_runs = 0
        self.params = params
        self.social_components = {
            soc: norm(soc, scale=soc//5)
            for soc in [10, 20, 30, 45, 60, 75, max_session_size + 1]
        }

    def reset(self):
        random_session = np.random.randint(0, self.unique_sessions.shape[0])
        
        user_to_run, session_to_run = self.unique_sessions.iloc[random_session][['user_id', 'session_30_count_raw']]
        self.current_session = self._get_events(user_to_run, session_to_run)
        self.metadata = self._metadata()
        self.current_session_index = 1
        self.reward = 0
        return self._state()
    
    def _row_to_dict(self, metadata):
        """
        Convert a row of metadata to a dictionary.
        """
        return metadata.to_dict()
    
    def _reward_exp(self, cum_session_event_raw):
        """
        Reward shaping as
            0 if cum_session_event_raw < size_cutoff
            (cum_session_event_raw - size_cutoff) * (cum_session_event_raw / size_cutoff) otherwise
        """
        
        if all([self.metadata['inc_small'] == 0, self.metadata['inc_medium'] == 0, self.metadata['inc_large'] == 0]):
            distance_shaper = cum_session_event_raw / 4
        
        else:
            distance_shaper = np.linalg.norm([
                self.metadata['inc_small'], 
                max(self.metadata['inc_medium'] - self.metadata['inc_small'], 0), 
                max(self.metadata['inc_large'] - self.metadata['inc_medium'], 0)
            ])
        if cum_session_event_raw <= self.metadata['size_cutoff']:
            return (cum_session_event_raw / self.metadata['size_cutoff']) * distance_shaper
        
        return (cum_session_event_raw - self.metadata['size_cutoff']) * (cum_session_event_raw / self.metadata['size_cutoff']) * distance_shaper

    def step(self, action):
        
        self._take_action(action)
        self._assign_social_components()
        next_state, done, meta = self._calculate_next_state()
        
        
        if done:
            current_session_index = self.current_session_index if \
                self.current_session_index != self.current_session.shape[0] else self.current_session.shape[0] - 1
            
            self.exp_runs += 1
            self.metadata['ended_event'] = self.current_session.iloc[current_session_index]['cum_session_event_raw']
            self.metadata['ended_time'] = self.current_session.iloc[current_session_index]['cum_session_time_raw']
            self.metadata['exp_runs'] = self.exp_runs
            self.episode_bins.append(self._row_to_dict(self.metadata))
           
            cum_session_event_raw = self.current_session.iloc[current_session_index]['cum_session_event_raw']
            reward_exp = self._reward_exp(cum_session_event_raw)
            
            return next_state, reward_exp , done, {}
        else:
            self.reward = self.current_session.iloc[self.current_session_index]['cum_session_event_raw']
            cum_session_event_raw = self.current_session.iloc[self.current_session_index]['cum_session_event_raw']
            
            reward_exp = self._reward_exp(cum_session_event_raw)
    
            self.current_session_index += 1        
            
            return next_state, reward_exp, done, meta
    
    def _assign_social_components(self):
        
        current_event = self.current_session_index
        social_components_keys = list(self.social_components.keys())
        social_likelihood_index = social_components_keys[bisect_left(social_components_keys, current_event)]
        
        if self.metadata[f'soc_{social_likelihood_index}'] > 0:
            return
        
        social_asssignmnet_fn = self.social_components[social_likelihood_index]
        assign_social = social_asssignmnet_fn.cdf(current_event)
         
        if all([assign_social >= .4, assign_social <= 7]):
            self.metadata[f'soc_{social_likelihood_index}'] = current_event

 
    def _metadata(self):
        session_metadata = self.current_session.iloc[0][RL_STAT_COLS].copy()
        session_metadata['ended'] = 0
        for meta_col in ['small', 'medium', 'large']:
            session_metadata[f'inc_{meta_col}'] = 0
            session_metadata[f'time_{meta_col}'] = 0
        
        for soc_col in self.social_components:
            session_metadata[f'soc_{soc_col}'] = 0

        return session_metadata
    
    def flush_episode_bins(self):
        episode_bins = self.episode_bins.copy()
        self.episode_bins = []
        return episode_bins
    
    def _calculate_next_state(self):
        
        if (self.current_session_index == self.current_session.shape[0]):
            return None, True, {}

        if self._continuing_in_session():
            return self._state(), False, {}
    
        return None, True, {}
         
    def _continuing_in_session(self):
        event_cutoff = self.current_session.iloc[self.current_session_index]['size_cutoff']
        current_session_event = self.current_session.iloc[self.current_session_index]['cum_session_event_raw']
        if current_session_event <= event_cutoff or current_session_event  >= MAX_EVAL_SIZE:
            return True
        
        param_mid = 0.1 if not self.params else self.params['mid']
        param_large = 0.2 if not self.params else self.params['large']
        param_window = 0.75 if not self.params else self.params['window']
    
        extending_low = self._probability_extending(current_session_event, self.metadata['inc_small']) - \
            (0.05 + np.random.normal(-0.02, 0.1, 100).mean())

            
        extending_medium = self._probability_extending(current_session_event, self.metadata['inc_medium']) - \
            (param_mid + np.random.normal(-0.02, 0.1, 100).mean()) 
            
        extending_large = self._probability_extending(current_session_event, self.metadata['inc_large']) - \
            (param_large + np.random.normal(-0.02, 0.1, 100).mean())
        
        max_social_component = max([self.metadata[f'soc_{i}'] for i in self.social_components])
        
        extending_social = self._probability_extending(current_session_event, max_social_component) - \
            (param_large + np.random.normal(-0.02, 0.1, 100).mean())
            
        return any([
            extending_low > 0.4 and extending_low <= param_window,
            extending_medium > 0.4 and extending_medium <= param_window,
            extending_large > 0.4 and extending_large <= param_window,
            extending_social > 0.4 and extending_social <= param_window
        ])
        
           
    
    def _probability_extending(self, current_session_event, incentive_event):
        if incentive_event == 0:
            return 0
         
        continue_session = norm(
            loc=max(incentive_event, 1),
            scale=max(incentive_event *.75, 1)
        ).cdf(max(current_session_event, 1)) 
        
        return continue_session
        

    def _get_events(self, user_id, session):
        subset = self.dataset[
            (self.dataset['user_id'] == user_id) &
            (self.dataset['session_30_count_raw'] == session).copy()
        ]

        subset = subset.sort_values(by=['date_time'])
        return subset
    
    def _take_action(self, action):
        
        if action == 0:
            return 1
        
        current_session_index = self.current_session_index if \
            self.current_session_index != self.current_session.shape[0] else self.current_session.shape[0] - 1
    
        if action == 1:
            if self.metadata['inc_small'] > 0:
                return 1

            self.metadata['inc_small'] = self.current_session.iloc[current_session_index]['cum_session_event_raw']
            self.metadata['time_small'] = self.current_session.iloc[current_session_index]['cum_session_time_raw']
            return 1
    
        elif action == 2:
            if self.metadata['inc_medium'] > 0:
                return 1
            self.metadata['inc_medium'] = self.current_session.iloc[current_session_index]['cum_session_event_raw']
            self.metadata['time_medium'] = self.current_session.iloc[current_session_index]['cum_session_time_raw']
            return 1
        
        else:
            if self.metadata['inc_large'] > 0:
                return 1
            self.metadata['inc_large'] = self.current_session.iloc[current_session_index]['cum_session_event_raw']
            self.metadata['time_large'] = self.current_session.iloc[current_session_index]['cum_session_time_raw']
            return 1

    def _state(self):

        if self.current_session_index > self.n_sequences:
            events = self.current_session.iloc[self.current_session_index - (self.n_sequences + 1):self.current_session_index][self.out_features]
            events['inc_small'] = self.metadata['inc_small']
            events['inc_medium'] = self.metadata['inc_medium']
            events['inc_large'] = self.metadata['inc_large']
            
            events = events.values
            
            
        else:
            
            delta = min((self.n_sequences + 1)- self.current_session_index, self.n_sequences)
            zero_cat = np.zeros((delta, len(self.out_features) + 3))
            events = self.current_session.iloc[:max(self.current_session_index, 1)][self.out_features]
            
            events['inc_small'] = self.metadata['inc_small']
            events['inc_medium'] = self.metadata['inc_medium']
            events['inc_large'] = self.metadata['inc_large']
           
            
            
            events = np.concatenate((zero_cat, events), axis=0)
        
        return events.astype(np.float32).T

In [9]:

import gym
import numpy as np
from scipy.stats import norm

MAX_EVAL_SIZE = 75


class CitizenScienceEnv(gym.Env):
    
    metadata = {'render.modes': ['human']}
    
    def __init__(self, dataset, out_features, n_sequences, params=None):
        """
        trajectories: dictionary of user_id to their respective trajectories.
        n_sequences: number of sequences used for preprocessing.
        n_features: number of features used for preprocessing.
        """
        super(CitizenScienceEnv, self).__init__()
        self.dataset = dataset
        self.unique_sessions = self.dataset[['user_id', 'session_30_count_raw']].drop_duplicates()
        self.n_sequences = n_sequences
        self.current_session = None
        self.current_session_index = 0
        self.reward = 0
        self.n_sequences = n_sequences
        self.out_features = out_features
        
        max_session_size = self.dataset['session_size'].max()
        
        self.action_space = gym.spaces.Discrete(4)
        self.observation_space = gym.spaces.Box(low=-1, high=91, shape=(len(out_features) + 3, n_sequences + 1), dtype=np.float32)
        self.episode_bins = []
        self.exp_runs = 0
        self.params = params
        

    def reset(self):
        random_session = np.random.randint(0, self.unique_sessions.shape[0])
        
        user_to_run, session_to_run = self.unique_sessions.iloc[random_session][['user_id', 'session_30_count_raw']]
        self.current_session = self._get_events(user_to_run, session_to_run)
        self.metadata = self._metadata()
        self.current_session_index = 1
        self.reward = 0
        return self._state()
    
    def _row_to_dict(self, metadata):
        """
        Convert a row of metadata to a dictionary.
        """
        return metadata.to_dict()
    
    def _reward_exp(self, cum_session_event_raw):
        """
        Reward shaping as
            0 if cum_session_event_raw < size_cutoff
            (cum_session_event_raw - size_cutoff) * (cum_session_event_raw / size_cutoff) otherwise
        """
        if cum_session_event_raw <= self.metadata['size_cutoff']:
            return cum_session_event_raw / self.metadata['size_cutoff']
        
        return (cum_session_event_raw - self.metadata['size_cutoff']) * (cum_session_event_raw / self.metadata['size_cutoff'])

    def step(self, action):
        
        self._take_action(action)
            
        next_state, done, meta = self._calculate_next_state()
        
        
        if done:
            current_session_index = self.current_session_index if \
                self.current_session_index != self.current_session.shape[0] else self.current_session.shape[0] - 1
            
            self.exp_runs += 1
            self.metadata['ended_event'] = self.current_session.iloc[current_session_index]['cum_session_event_raw']
            self.metadata['ended_time'] = self.current_session.iloc[current_session_index]['cum_session_time_raw']
            self.metadata['exp_runs'] = self.exp_runs
            self.episode_bins.append(self._row_to_dict(self.metadata))
            
            self.metadata['ended_event'] = self.current_session.iloc[current_session_index]['cum_session_event_raw']
            self.metadata['ended_time'] = self.current_session.iloc[current_session_index]['cum_session_time_raw']
            self.metadata['exp_runs'] = self.exp_runs
            self.episode_bins.append(self._row_to_dict(self.metadata))
           
            cum_session_event_raw = self.current_session.iloc[current_session_index]['cum_session_event_raw']
            reward_exp = self._reward_exp(cum_session_event_raw)
            
            return next_state, reward_exp , done, {}
        else:
            self.reward = self.current_session.iloc[self.current_session_index]['cum_session_event_raw']
            cum_session_event_raw = self.current_session.iloc[self.current_session_index]['cum_session_event_raw']
            
            reward_exp = self._reward_exp(cum_session_event_raw)
    
            self.current_session_index += 1        
            
            return next_state, reward_exp, done, meta
    
    def _metadata(self):
        session_metadata = self.current_session.iloc[0][RL_STAT_COLS].copy()
        session_metadata['ended'] = 0
        for meta_col in ['small', 'medium', 'large']:
            session_metadata[f'inc_{meta_col}'] = 0
            session_metadata[f'time_{meta_col}'] = 0

        return session_metadata
    
    def flush_episode_bins(self):
        episode_bins = self.episode_bins.copy()
        self.episode_bins = []
        return episode_bins
    
    def _calculate_next_state(self):
        
        if (self.current_session_index == self.current_session.shape[0]):
            return None, True, {}

        if self._continuing_in_session():
            return self._state(), False, {}
    
        return None, True, {}
         
    def _continuing_in_session(self):
        event_cutoff = self.current_session.iloc[self.current_session_index]['size_cutoff']
        current_session_event = self.current_session.iloc[self.current_session_index]['cum_session_event_raw']
        if current_session_event <= event_cutoff or current_session_event  >= MAX_EVAL_SIZE:
            return True
        
        param_mid = 0.1 if not self.params else self.params['mid']
        param_large = 0.2 if not self.params else self.params['large']
        param_window = 0.75 if not self.params else self.params['window']
    
        extending_low = self._probability_extending(current_session_event, self.metadata['inc_small']) - \
            (0.05 + np.random.normal(-0.02, 0.1, 100).mean())

            
        extending_medium = self._probability_extending(current_session_event, self.metadata['inc_medium']) - \
            (param_mid + np.random.normal(-0.02, 0.1, 100).mean()) 
            
        extending_large = self._probability_extending(current_session_event, self.metadata['inc_large']) - \
            (param_large + np.random.normal(-0.02, 0.1, 100).mean())
            
        return any([
            extending_low > 0.4 and extending_low <= param_window,
            extending_medium > 0.4 and extending_medium <= param_window,
            extending_large > 0.4 and extending_large <= param_window
        ])
        
           
    
    def _probability_extending(self, current_session_event, incentive_event):
        if incentive_event == 0:
            return 0
         
        continue_session = norm(
            loc=max(incentive_event, 1),
            scale=max(incentive_event *.75, 1)
        ).cdf(max(current_session_event, 1)) 
        
        return continue_session
        

    def _get_events(self, user_id, session):
        subset = self.dataset[
            (self.dataset['user_id'] == user_id) &
            (self.dataset['session_30_count_raw'] == session).copy()
        ]

        subset = subset.sort_values(by=['date_time'])
        return subset
    
    def _take_action(self, action):
        if action == 0:
            return 1
        
        current_session_index = self.current_session_index if \
            self.current_session_index != self.current_session.shape[0] else self.current_session.shape[0] - 1
    
        if action == 1:
            if self.metadata['inc_small'] > 0:
                return 1

            self.metadata['inc_small'] = self.current_session.iloc[current_session_index]['cum_session_event_raw']
            self.metadata['time_small'] = self.current_session.iloc[current_session_index]['cum_session_time_raw']
            return 1
    
        elif action == 2:
            if self.metadata['inc_medium'] > 0:
                return 1
            self.metadata['inc_medium'] = self.current_session.iloc[current_session_index]['cum_session_event_raw']
            self.metadata['time_medium'] = self.current_session.iloc[current_session_index]['cum_session_time_raw']
            return 1
        
        else:
            if self.metadata['inc_large'] > 0:
                return 1
            self.metadata['inc_large'] = self.current_session.iloc[current_session_index]['cum_session_event_raw']
            self.metadata['time_large'] = self.current_session.iloc[current_session_index]['cum_session_time_raw']
            return 1

    def _state(self):

        if self.current_session_index > self.n_sequences:
            events = self.current_session.iloc[self.current_session_index - (self.n_sequences + 1):self.current_session_index][self.out_features]
            events['inc_small'] = self.metadata['inc_small']
            events['inc_medium'] = self.metadata['inc_medium']
            events['inc_large'] = self.metadata['inc_large']
            
            events = events.values
            
            
        else:
            
            delta = min((self.n_sequences + 1)- self.current_session_index, self.n_sequences)
            zero_cat = np.zeros((delta, len(self.out_features) + 3))
            events = self.current_session.iloc[:max(self.current_session_index, 1)][self.out_features]
            
            events['inc_small'] = self.metadata['inc_small']
            events['inc_medium'] = self.metadata['inc_medium']
            events['inc_large'] = self.metadata['inc_large']
            
            
            events = np.concatenate((zero_cat, events), axis=0)
        
        return events.astype(np.float32).T

In [10]:
# %load rl_results_eval_cpu.py
import argparse
import logging
import os
from datetime import datetime
from functools import reduce
from pprint import pformat
from typing import Callable
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.dqn.policies import CnnPolicy
import boto3
import random
import numpy as np
import pandas as pd
import torch
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor

from stable_baselines3 import DQN, PPO, A2C, SAC, TD3
from sb3_contrib import QRDQN

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)
np.set_printoptions(precision=4, linewidth=200, suppress=True)
torch.set_printoptions(precision=2, linewidth=200, sci_mode=False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
import zipfile
import torch.nn as nn

S3_BASELINE_PATH = 'dissertation-data-dmiller'
N_SEQUENCES = 15
CHECKPOINT_FREQ = 100_000
TB_LOG = 10_000
WINDOW = 2
import glob
TB_LOG = 10_000
WINDOW = 1
REWARD_CLIP = 90
MIN_MAX_RANGE = (10, 90)

MODEL_DICT = {
    'dqn': DQN,
    'qrdqn': QRDQN,
    'a2c': A2C,
}


global logger

logger = logging.getLogger('rl_results_eval')
logger.setLevel(logging.INFO)
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
import re

def parse_args():
    parse = argparse.ArgumentParser()
    parse.add_argument('--write_path', type=str, default='rl_evaluation')
    parse.add_argument('--part', type=str, default='eval')
    parse.add_argument('--algo', type=str, default='dqn_pred_cnn'),

    parse.add_argument('--run_date', type=str, default='2023-06-13_16-11-42'),
    parse.add_argument('--n_files', type=int, default=30)
                       
    return parse.parse_args()


def find_s3_candidate(algo, run_date):
    
    folder_prefix = os.path.join(
        'experiments',
        "q2",
        algo,
        run_date,
        'checkpoints'
    )

    
    logger.info(f'Looking for files in {folder_prefix}')
    
    files = [
        {
            'key': file['Key'],
            'last_modified': file['LastModified'],
            'check_index': int(re.sub('[^0-9]', '', file['Key'].split('/')[-1]))
        }
        for file in client.list_objects_v2(Bucket=S3_BASELINE_PATH, Prefix=folder_prefix)['Contents']
    ]
    
    s3_candidate = sorted(files, key=lambda x: x['check_index'])[-1]['key']
    


    
    logger.info(f'Found candiate: {s3_candidate}')
    
    return s3_candidate

def get_policy(algo, run_date):
    
    
    s3_candidate = find_s3_candidate(algo, run_date)
    if not os.path.exists(os.path.dirname(s3_candidate)):
        logger.info(f'Creating policy path {os.path.dirname(s3_candidate)}')
        
        os.makedirs(os.path.dirname(s3_candidate), exist_ok=True)
       
    # return s3_candidate 
    client.download_file(S3_BASELINE_PATH, s3_candidate, s3_candidate)
    return s3_candidate
        

def simplify_experiment(vectorized_df):
    vectorized_df = [
        df[(df['session_size'] >= MIN_MAX_RANGE[0]) & (df['session_size'] <= MIN_MAX_RANGE[1])] for df in vectorized_df
    ]

    return vectorized_df

      
def download_dataset_from_s3(client, base_read_path, full_read_path):
    logger.info(f'Downloading data from {base_read_path}')
    os.makedirs(base_read_path, exist_ok=True)
    
    logger.info(f'Downloading data from dissertation-data-dmiller/{full_read_path}')
    client.download_file(
        'dissertation-data-dmiller',
        full_read_path,
        full_read_path
    )
    logger.info(f'Downloaded data from dissertation-data-dmiller/{full_read_path}')
    

def simplify_experiment(vectorized_df):
    vectorized_df = [
        df[(df['session_size'] >= MIN_MAX_RANGE[0]) & (df['session_size'] <= MIN_MAX_RANGE[1])] for df in vectorized_df
    ]

    return vectorized_df

def rebatch_data(vectorized_df):
    df_sublist = []
    for i in range(0, len(vectorized_df), 5):
        df_sublist.append(pd.concat(vectorized_df[i:i+5], ignore_index=True))
    return df_sublist

def _label_or_pred(algo):
    if 'label' in algo:
        return 'label'
    elif 'pred' in algo:
        return 'pred'
    else:
        return None


def main(args):
    
    global client
    client = boto3.client('s3')


    logger.info('Starting offlline evaluation of RL model')
    
    write_path, part, algo, run_date, n_files, eval_episodes, q2, model, reward_fx = (
        args.write_path,
        args.part,
        args.algo,
        args.run_date,
        args.n_files,
        args.eval_episodes,
        args.q2,
        args.model,
        args.reward_fx
    )
    
    
    read_path = os.path.join(
        'rl_ready_data_conv',
        f'files_used_{n_files}',
        'window_1',
        f'batched_{part}'
    )
   
    logger.info(f'Reading from {read_path}, writing to {write_path}') 
    files_to_read = glob.glob(os.path.join(read_path, '*.parquet'))
    logger.info(f'Found {len(files_to_read)} files to read')
    
    
    
    feature_cols = FEATURE_COLUMNS + [_label_or_pred(algo)] if _label_or_pred(algo) else FEATURE_COLUMNS
   
    logger.info(f'n features: {len(feature_cols)}')

    env_files = [
        pd.read_parquet(file) for file in files_to_read
    ]


    logger.info(f'Loaded env files: clipping to {MIN_MAX_RANGE}')
    
    env_files = simplify_experiment(env_files)
    
    if 'model' == 'a2c':
        logger.info('rebatching data')
        env_files = rebatch_data(env_files)
        
    vec_env = DummyVecEnv([lambda: CitizenScienceEnvQ2(df, feature_cols, N_SEQUENCES) for df in env_files])
   

    
    tensorboard_dir = os.path.join(
        args.write_path,
        f'{args.part}/{algo}_{run_date}'
    )
       
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
 
    logger.info(f'Logging to {tensorboard_dir}, n envs: {len(env_files)}: device: {device}')
    
    monitor_env = VecMonitor(vec_env)
    policy_path = get_policy(algo, run_date)
    model_proto = MODEL_DICT[model]
    logger.info(f'Loading model from {policy_path}: {model_proto}')
    model = model_proto.load(policy_path, env=monitor_env, tensorboard_log=tensorboard_dir, device=device)
    logger.info(f'Running evaluation: n_episodes: {eval_episodes}')
    evaluate_policy(
        model,
        model.get_env(),
        deterministic=False,
        n_eval_episodes=eval_episodes,
       
    )
    
    logger.info(f'Finished evaluation getting attributes')
    dist_list = model.get_env().get_attr('episode_bins')
    values_to_log = [item for sublist in dist_list for item in sublist if len(sublist) > 0]
    out_files = pd.DataFrame(values_to_log)
    logger.info(f'Attributes: {out_files.shape}')
    write_path = os.path.join(
        write_path,
        part,
        "q2" if q2 else "q1",
        f'{algo}_{reward_fx}_{eval_episodes}.parquet'
    )
    if not os.path.exists(os.path.dirname(write_path)):
        logger.info(f'Creating write path {os.path.dirname(write_path)}')
        os.makedirs(os.path.dirname(write_path), exist_ok=True)
        
    logger.info(f'Writing to {write_path}')
    out_files.to_parquet(write_path)

In [11]:
class TrainArg:
    write_path = 'rl_evaluation'
    part = 'train'
    algo = 'dqn_pred_cnn'
    model = 'dqn'
    run_date = '2023-07-20_09-18-23'
    n_files = 30
    eval_episodes = 25_000
    q2 = True
    reward_fx = 'dist_enc_enforce_baseline'
    
class EvalArg:
    write_path = 'rl_evaluation'
    part = 'eval'
    algo = 'dqn_pred_cnn'
    model = 'dqn'
    run_date = '2023-07-20_09-18-23'
    n_files = 30
    eval_episodes = 15_000
    q2 = True
    reward_fx = 'dist_enc_enforce_baseline'
    
main(EvalArg)
main(TrainArg)

07/20/2023 02:20:34 PM Found credentials in environment variables.


07/20/2023 02:20:34 PM Starting offlline evaluation of RL model
07/20/2023 02:20:34 PM Reading from rl_ready_data_conv/files_used_30/window_1/batched_eval, writing to rl_evaluation
07/20/2023 02:20:34 PM Found 100 files to read
07/20/2023 02:20:34 PM n features: 23
07/20/2023 02:20:38 PM Loaded env files: clipping to (10, 90)
/usr/local/lib/python3.9/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause i